In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from tensorflow.keras import layers
from tensorflow import keras
from keras.constraints import maxnorm
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,  Flatten, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import metrics
from keras.metrics import AUC
from keras.metrics import Precision
from keras.metrics import Recall

from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import sklearn
from sklearn.ensemble import StackingClassifier
import pickle

# Data Preprocessing

In [ ]:
X_train_trf1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_train_trf1_v3.csv')
X_val_trf1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_val_trf1_v3.csv')
X_test_trf1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_test_trf1_v3.csv')

#Drop rating_month and rating_year columns
#Drop primaryTitle, userId_ori, movieId_ori

X_test_complete = X_test_trf1.loc[:,['primaryTitle', 'userId_ori']]

def dropRatingDate(df):
  cols = [0, 1, 2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 42]
  return df.drop(df.columns[cols], axis=1)

X_train_trf1 = dropRatingDate(X_train_trf1)
X_test_trf1 = dropRatingDate(X_test_trf1)
X_val_trf1 = dropRatingDate(X_val_trf1)

columns_list = X_train_trf1.columns.tolist()

X_train_trf1 = X_train_trf1.to_numpy()
X_val_trf1 = X_val_trf1.to_numpy()
X_test_trf1 = X_test_trf1.to_numpy() 

y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y_train_xg_v3.csv').to_numpy(dtype = 'int')
y_val= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y_val_xg_v3.csv').to_numpy(dtype = 'int')
y_test= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y_test_xg_v3.csv').to_numpy(dtype = 'int')

In [ ]:
#Convert 9bins to 2bins
def nineToTwoBins(y_dataset):
    for i in range(0, len(y_dataset)): 
        if y_dataset[i] < 7:
            y_dataset[i] = 0
        else:
            y_dataset[i] = 1
    return y_dataset

In [ ]:
nineToTwoBins(y_train)
nineToTwoBins(y_val)
nineToTwoBins(y_test)

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
training_data = {'X_train':X_train_trf1,'Y_train':y_train,
                'X_val': X_val_trf1,'Y_val':y_val,
                'X_test': X_test_trf1 ,'Y_test':y_test}

# Stacking and Loading Optimized Models

**Build Model Function based on Bayesian Optimized Model**

In [ ]:
def build_model():
  # create model
  model = Sequential()

  # Define input layer, and first hidden layer
  # neurons for first hidden layer
  model.add(Dense(970, input_dim=27, activation='relu')
  )

  # Define 2nd hidden (Dense) layers onward
  # Tune layers, neurons
  for i, neuron in enumerate([970, 970, 20, 970, 970,
                              970, 970, 970, 970, 970,
                              970, 970], start = 2):
    model.add(Dense(units=neuron,
                    activation='relu', 
                    name=f'Hidden{i}'))
    # Add drop out layers to 5th hidden layer
    if i == 4:
      model.add(Dropout(rate=0.25)
      )

  # Define output layer
  model.add(Dense(1, activation='sigmoid')
  )

  # Compile model
  # Tune learning rate at optimizer
  model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=[AUC(),Precision(), Recall()]
                  )
  return model

**XGBClassifier Based on GridSearch Tuned Model**

In [ ]:
XG_clf = XGBClassifier(learning_rate=0.1,
                    n_estimators=1000,
                    max_depth=9,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.9,
                    colsample_bytree=0.6,
                    reg_alpha = 0.01,
                    tree_method = "gpu_hist",
                    objective='multi:softmax',
                    num_class=2,
                    seed=27)

XG_clf._estimator_type = "classifier"

NN_clf = KerasClassifier(build_fn=build_model, 
                          batch_size = 500,
                          epochs = 80)
NN_clf._estimator_type = "classifier"

estimators= [('XG', XG_clf), ('NN', NN_clf) ]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), stack_method='predict_proba')

clf.fit(X_train_trf1, y_train)
print("Stacking model score: %.3f" % clf.score(X_test_trf1, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/80
466/466 [==============================] - 7s 13ms/step - loss: 0.6185 - auc_4: 0.5817 - precision_4: 0.4710 - recall_4: 0.0378
Epoch 2/80
466/466 [==============================] - 6s 13ms/step - loss: 0.5917 - auc_4: 0.6546 - precision_4: 0.5700 - recall_4: 0.1862
Epoch 3/80
466/466 [==============================] - 6s 12ms/step - loss: 0.5873 - auc_4: 0.6666 - precision_4: 0.5680 - recall_4: 0.2102
Epoch 4/80
466/466 [==============================] - 6s 12ms/step - loss: 0.5842 - auc_4: 0.6710 - precision_4: 0.5826 - recall_4: 0.2049
Epoch 5/80
466/466 [==============================] - 6s 12ms/step - loss: 0.5793 - auc_4: 0.6799 - precision_4: 0.5773 - recall_4: 0.2219
Epoch 6/80
466/466 [==============================] - 6s 12ms/step - loss: 0.5744 - auc_4: 0.6907 - precision_4: 0.5861 - recall_4: 0.2403
Epoch 7/80
466/466 [==============================] - 6s 13ms/step - loss: 0.5719 - auc_4: 0.6959 - precision_4: 0.5907 - recall_4: 0.2429
Epoch 8/80
466/466 [=======

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/80
373/373 [==============================] - 6s 13ms/step - loss: 0.6164 - auc_6: 0.5885 - precision_6: 0.4648 - recall_6: 0.0478
Epoch 2/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5900 - auc_6: 0.6598 - precision_6: 0.5668 - recall_6: 0.1932
Epoch 3/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5880 - auc_6: 0.6675 - precision_6: 0.5597 - recall_6: 0.2241
Epoch 4/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5846 - auc_6: 0.6756 - precision_6: 0.5744 - recall_6: 0.2336
Epoch 5/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5775 - auc_6: 0.6857 - precision_6: 0.5731 - recall_6: 0.2435
Epoch 6/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5763 - auc_6: 0.6891 - precision_6: 0.5909 - recall_6: 0.2319
Epoch 7/80
373/373 [==============================] - 5s 13ms/step - loss: 0.5751 - auc_6: 0.6925 - precision_6: 0.5932 - recall_6: 0.2444
Epoch 8/80
373/373 [=======

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Stacking model score: 0.799


In [ ]:
predictions = clf.predict(X_test_trf1)
predictions

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
predictions = clf.predict(X_test_trf1)

X_test_complete['prediction'] = predictions

X_test_complete = X_test_complete[X_test_complete.prediction != 0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [ ]:
X_test_complete

,primaryTitle,userId_ori,prediction
1,True Grit,1920,1
2,Secretary,159816,1
3,Gravity,30643,1
6,Toy Story,74429,1
13,Out of Africa,2177,1
...,...,...,...
68426,Superman,2177,1
68427,500 Days of Summer,39905,1
68430,Cape Fear,103611,1
68431,Whiplash,94154,1


In [ ]:
def recommend(userid):
  recommend = []
  for i in range(0, X_test_complete.shape[0] - 1):
    if X_test_complete.iloc[i].userId_ori == userid:
      recommend.append(X_test_complete.iloc[i].primaryTitle)
  return pd.DataFrame(data=recommend, columns = ['Recommended_Movies'])

In [ ]:
#return recommendation list for userId 1920
recommend(1920)

,Recommended_Movies
0,True Grit
1,Contact
2,Roman Holiday
3,Mary Shelley's Frankenstein
4,Pinocchio
5,Blade Runner 2049
6,Peter Pan
7,Mary Poppins
8,Black Panther
9,Vertigo
